In [111]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [127]:
import pandas as pd

tweets = pd.read_csv("siads-699/data/tweets_with_sentiment_and_topic.csv")
tweets['handle'] = tweets['handle'].apply(lambda x: f"@{x}" if not str(x).startswith('@') else x)

tweets.head()

,Unnamed: 0,id,user_id,handle,clean_text,lang,timestamp,sentiment_label,sentiment_score,top_topic,Cryptocurrency and Fintech,Technology,Healthcare and Pharmaceuticals,Defense and Aerospace,Financials,Airlines and Travel,Energy,Agriculture,Automotive
0,0,1603456812723933194,12.0,@jack,"RT @ODELL: @nobsbitcoin love to see it, if yal...",en,2022-12-15 18:27:49.597000+00:00,Neutral,0.977740,Cryptocurrency and Fintech,0.757990,0.376340,0.045903,0.032895,0.019666,0.019118,0.013074,0.010588,0.007226
1,1,1600412216687149056,12.0,@jack,RT @TheGuySwann: It would be so amazing to see...,en,2022-12-07 08:49:41.336000+00:00,Positive,0.999987,Cryptocurrency and Fintech,0.868653,0.853358,0.025603,0.044147,0.210190,0.011476,0.110531,0.037469,0.007175
2,2,1595502702854692866,12.0,@jack,RT @kendricklamar: big steppers tour (direct...,en,2022-11-23 19:41:02.024000+00:00,Neutral,0.999613,Technology,0.012002,0.252524,0.057174,0.072260,0.034317,0.007996,0.006217,0.003851,0.017827
3,3,1589450807216599040,12.0,@jack,@litcapital @elonmusk Sure,en,2022-11-07 02:52:57.665000+00:00,Neutral,0.985752,Technology,0.284053,0.768173,0.263169,0.119607,0.364800,0.058842,0.239539,0.033245,0.125718
4,4,1582442966521503745,12.0,@jack,RT @bluesky: Bluesky is building a social prot...,en,2022-10-18 18:46:18.244000+00:00,Neutral,0.999923,Technology,0.000134,0.058334,0.002962,0.001737,0.000540,0.004032,0.008653,0.003117,0.000706


In [128]:
print(tweets['handle'].unique())

['@jack' '@realDonaldTrump' '@DavidSacks' '@nntaleb' '@paulkrugman'
 '@elonmusk' '@pmarca' '@POTUS' '@SBF_FTX' '@VivekGRamaswamy'
 '@richardbranson' '@JoeBiden' '@SenWarren']


In [146]:
name_handle_map = {
    "Jack Dorsey": "@jack",
    "Donald J. Trump": "@realDonaldTrump",
    "David Sacks": "@DavidSacks",
    "Nassim Nicholas Taleb": "@nntaleb",
    "Paul Krugman": "@paulkrugman",
    "Elon Musk": "@elonmusk",
    "Marc Andreessen": "@pmarca",
    "President of the United States": "@POTUS",
    "Sam Bankman-Fried": "@SBF_FTX",
    "Vivek Ramaswamy": "@VivekGRamaswamy",
    "Richard Branson": "@richardbranson",
    "Joe Biden": "@JoeBiden",
    "Elizabeth Warren": "@SenWarren"
}


In [147]:
name_handle_df = pd.DataFrame(name_handle_map.items(), columns=["name", "handle"])
name_handle_df.head()

,name,handle
0,Jack Dorsey,@jack
1,Donald J. Trump,@realDonaldTrump
2,David Sacks,@DavidSacks
3,Nassim Nicholas Taleb,@nntaleb
4,Paul Krugman,@paulkrugman


In [148]:
related_stocks = pd.read_csv("siads-699/data/04_people_stock_link.csv", sep=';')
related_stocks.head()

,person name,industry,company name,stock exchange,ticker,date range,description of link
0,Buff Mage,Conglomerate,Berkshire Hathaway,NYSE,BRK.B,08-1965 -,"Often compared to Warren Buffett, invests in c..."
1,Buff Mage,Technology,Apple,NASDAQ,AAPL,08-2016 -,Holds significant tech investments
2,Buff Mage,Banking,Bank of America,NYSE,BAC,08-2011 -,Known to invest heavily in major banks
3,Buff Mage,Beverages,Coca-Cola,NYSE,KO,08-1988 -,Long-standing soft drink holdings
4,Buff Mage,Financial Services,American Express,NYSE,AXP,08-1963 -,Significant interest in credit services


In [149]:
print(related_stocks['person name'].unique())

['Buff Mage' 'David Sacks' 'Donald J. Trump' 'Elizabeth Warren'
 'Elon Musk' 'Joe Biden' 'Jung Musk' 'Marc Andreessen' 'Mr. Tweet'
 'Nassim Nicholas Taleb' 'Naughtius Maximus' 'Paul Krugman'
 'Richard Branson' 'SBF' 'Sphere-Earthers Rūl' 'Vivek Ramaswamy' 'jack']


In [150]:
elon_aliases = [
    'Buff Mage',
    'Jung Musk',
    'Mr. Tweet',
    'Naughtius Maximus',
    'Sphere-Earthers Rūl'
]

# Replace aliases in the 'person name' column with 'Elon Musk'
related_stocks['person name'] = related_stocks['person name'].replace(elon_aliases, 'Elon Musk')

related_stocks.head()

,person name,industry,company name,stock exchange,ticker,date range,description of link
0,Elon Musk,Conglomerate,Berkshire Hathaway,NYSE,BRK.B,08-1965 -,"Often compared to Warren Buffett, invests in c..."
1,Elon Musk,Technology,Apple,NASDAQ,AAPL,08-2016 -,Holds significant tech investments
2,Elon Musk,Banking,Bank of America,NYSE,BAC,08-2011 -,Known to invest heavily in major banks
3,Elon Musk,Beverages,Coca-Cola,NYSE,KO,08-1988 -,Long-standing soft drink holdings
4,Elon Musk,Financial Services,American Express,NYSE,AXP,08-1963 -,Significant interest in credit services


In [151]:
print(related_stocks['person name'].unique())

['Elon Musk' 'David Sacks' 'Donald J. Trump' 'Elizabeth Warren'
 'Joe Biden' 'Marc Andreessen' 'Nassim Nicholas Taleb' 'Paul Krugman'
 'Richard Branson' 'SBF' 'Vivek Ramaswamy' 'jack']


In [152]:
related_stocks = related_stocks.rename(columns={
    'person name': 'name',
    'ticker': 'ticker'
})

# Merge in handles
related_stocks = related_stocks.merge(name_handle_df, how='inner', on='name')
related_stocks.head()


,name,industry,company name,stock exchange,ticker,date range,description of link,handle
0,Elon Musk,Conglomerate,Berkshire Hathaway,NYSE,BRK.B,08-1965 -,"Often compared to Warren Buffett, invests in c...",@elonmusk
1,Elon Musk,Technology,Apple,NASDAQ,AAPL,08-2016 -,Holds significant tech investments,@elonmusk
2,Elon Musk,Banking,Bank of America,NYSE,BAC,08-2011 -,Known to invest heavily in major banks,@elonmusk
3,Elon Musk,Beverages,Coca-Cola,NYSE,KO,08-1988 -,Long-standing soft drink holdings,@elonmusk
4,Elon Musk,Financial Services,American Express,NYSE,AXP,08-1963 -,Significant interest in credit services,@elonmusk


In [153]:
tweets_expanded = tweets.merge(related_stocks[['handle', 'ticker']], on='handle', how='inner')
tweets_expanded.head()

,Unnamed: 0,id,user_id,handle,clean_text,lang,timestamp,sentiment_label,sentiment_score,top_topic,Cryptocurrency and Fintech,Technology,Healthcare and Pharmaceuticals,Defense and Aerospace,Financials,Airlines and Travel,Energy,Agriculture,Automotive,ticker
0,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,0.006034,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,DWAC
1,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,0.006034,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,LVS
2,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,0.006034,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,MGM
3,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,0.006034,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,WYNN
4,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,0.006034,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,FOX


In [154]:
available_tickers = os.listdir("siads-699/data/taq")
print(available_tickers)
os.listdir("siads-699/data/taq/SYM_ROOT=PYPL")

['SYM_ROOT=SPOT', 'SYM_ROOT=DIS', 'SYM_ROOT=WFC', 'SYM_ROOT=BAH', 'SYM_ROOT=TTWO', 'SYM_ROOT=MMS', 'SYM_ROOT=WYNN', 'SYM_ROOT=NOK', 'SYM_ROOT=ADBE', 'SYM_ROOT=ATVI', 'SYM_ROOT=ZM', 'SYM_ROOT=LDOS', 'SYM_ROOT=CVX', 'SYM_ROOT=DELL', 'SYM_ROOT=F', 'SYM_ROOT=GME', 'SYM_ROOT=MSTR', 'SYM_ROOT=PYPL', 'SYM_ROOT=AMZN', 'SYM_ROOT=KO', 'SYM_ROOT=GE', 'SYM_ROOT=NVDA', 'SYM_ROOT=PFE', 'SYM_ROOT=GS', 'SYM_ROOT=COIN', 'SYM_ROOT=C', 'SYM_ROOT=PEP', 'SYM_ROOT=MRNA', 'SYM_ROOT=SQ', 'SYM_ROOT=TGT', 'SYM_ROOT=GM', 'SYM_ROOT=DWAC', 'SYM_ROOT=SHOP', 'SYM_ROOT=MSFT', 'SYM_ROOT=NVAX', 'SYM_ROOT=AMC', 'SYM_ROOT=TWTR', 'SYM_ROOT=SNAP', 'SYM_ROOT=NKE', 'SYM_ROOT=BYND', 'SYM_ROOT=XOM', 'SYM_ROOT=PINS', 'SYM_ROOT=AMD', 'SYM_ROOT=BAC', 'SYM_ROOT=AAPL', 'SYM_ROOT=MGM', 'SYM_ROOT=SPCE', '.stamp', 'SYM_ROOT=TSM', 'SYM_ROOT=LMT', 'SYM_ROOT=CAT', 'SYM_ROOT=AAL', 'SYM_ROOT=NOC', 'SYM_ROOT=FOX', 'SYM_ROOT=PLTR', 'SYM_ROOT=WMT', 'SYM_ROOT=NFLX', 'SYM_ROOT=HD', 'SYM_ROOT=BA', 'SYM_ROOT=JNJ', 'SYM_ROOT=TSLA', 'SYM_ROOT=DAL',

['year=2017', 'year=2015', 'year=2020', 'year=2016', 'year=2018', 'year=2019']

In [155]:
!pip install pyarrow

In [156]:
import glob
import re

# Get all available ticker-year combos from folders
parquet_paths = glob.glob("siads-699/data/taq/SYM_ROOT=*/year=*")
available_pairs = [
    (re.search(r"SYM_ROOT=(.*?)/year=(\d+)", path).groups()[0], int(re.search(r"year=(\d+)", path).group(1)))
    for path in parquet_paths if 'year=' in path
]
available_set = set(available_pairs)


In [157]:
from dateutil import parser
import pandas as pd

# Custom parse function that safely parses each timestamp
def safe_parse(ts):
    try:
        return parser.parse(ts)
    except Exception:
        return pd.NaT

# Apply the custom function
tweets_expanded['year'] = tweets_expanded['timestamp'].apply(safe_parse).dt.year
tweets_expanded.head()

,Unnamed: 0,id,user_id,handle,clean_text,lang,timestamp,sentiment_label,sentiment_score,top_topic,...,Technology,Healthcare and Pharmaceuticals,Defense and Aerospace,Financials,Airlines and Travel,Energy,Agriculture,Automotive,ticker,year
0,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,...,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,DWAC,2021
1,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,...,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,LVS,2021
2,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,...,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,MGM,2021
3,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,...,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,WYNN,2021
4,36,1346954970910707712,25073877.0,@realDonaldTrump,These are the things and events that happen wh...,en,2021-01-06 23:01:04.935000+00:00,Neutral,0.999843,Energy,...,0.007289,0.005461,0.006295,0.011067,0.001142,0.078936,0.002344,0.002762,FOX,2021


In [158]:
def process_single_tweet(row, stock_base_path):
    from datetime import timedelta
    import pandas as pd
    import glob
    import os

    try:
        ticker = row['ticker']
        tweet_time = pd.to_datetime(row['timestamp']).tz_localize(None)
        year = tweet_time.year

        # Path to stock data files
        partition_path = os.path.join(stock_base_path, f"SYM_ROOT={ticker}", f"year={year}", "*.parquet")
        files = glob.glob(partition_path)

        if not files:
            print(f" No files for {ticker} {year}")
            return pd.DataFrame()

        # 30-minute window around the tweet time
        start = tweet_time - timedelta(minutes=30)
        end = tweet_time + timedelta(minutes=30)

        # Single pull for data 24 hours after the tweet
        pull_24h_after = tweet_time + timedelta(days=1)

        all_windows = []
        for file in files:
            try:
                df = pd.read_parquet(file)

                # Ensure timestamp is datetime
                if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
                    df['timestamp'] = pd.to_datetime(df['timestamp'])

                if 'avg_price' in df.columns and 'price' not in df.columns:
                    df = df.rename(columns={'avg_price': 'price'})

                if 'price' not in df.columns or 'timestamp' not in df.columns:
                    continue

                # Filter for ±30 minute window around tweet time
                mask_window = (df['timestamp'] >= start) & (df['timestamp'] <= end)
                window = df.loc[mask_window].copy()

                # Pull data exactly 24 hours after the tweet
                mask_24h_after = (df['timestamp'] >= pull_24h_after) & (df['timestamp'] < pull_24h_after + timedelta(minutes=30))
                data_24h_after = df.loc[mask_24h_after].copy()

                # Combine the 30-minute window and 24-hour after data
                if not window.empty:
                    window['tweet_id'] = row['id']
                    window['handle'] = row['handle']
                    window['ticker'] = ticker
                    window['sentiment_label'] = row['sentiment_label']
                    window['sentiment_score'] = row['sentiment_score']
                    window['top_topic'] = row.get('top_topic')  # new

                    # Add sector scores if available
                    sector_cols = [
                        'Technology', 'Healthcare and Pharmaceuticals', 'Defense and Aerospace',
                        'Financials', 'Airlines and Travel', 'Energy', 'Agriculture', 'Automotive'
                    ]
                    for col in sector_cols:
                        if col in row:
                            window[col] = row[col]

                    window['minutes_since_tweet'] = (window['timestamp'] - tweet_time).dt.total_seconds() / 60
                    all_windows.append(window)

                if not data_24h_after.empty:
                    data_24h_after['tweet_id'] = row['id']
                    data_24h_after['handle'] = row['handle']
                    data_24h_after['ticker'] = ticker
                    data_24h_after['sentiment_label'] = row['sentiment_label']
                    data_24h_after['sentiment_score'] = row['sentiment_score']
                    data_24h_after['top_topic'] = row.get('top_topic')  # new

                    # Add sector scores if available
                    for col in sector_cols:
                        if col in row:
                            data_24h_after[col] = row[col]

                    data_24h_after['minutes_since_tweet'] = (data_24h_after['timestamp'] - tweet_time).dt.total_seconds() / 60
                    all_windows.append(data_24h_after)

            except Exception as e:
                print(f"Error reading {file}: {e}")
                continue

        return pd.concat(all_windows, ignore_index=True) if all_windows else pd.DataFrame()

    except Exception as e:
        print(f"Failed tweet {row.get('id')}: {e}")
        return pd.DataFrame()

In [159]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def process_tweets_parallel(tweets_df, stock_base_path, checkpoint_dir, batch_size=100, max_workers=4):
    os.makedirs(checkpoint_dir, exist_ok=True)
    tweets_df = tweets_df.reset_index(drop=True)

    # Load already processed tweet IDs
    done_ids = set()
    for file in os.listdir(checkpoint_dir):
        if file.endswith(".parquet"):
            try:
                df = pd.read_parquet(os.path.join(checkpoint_dir, file), columns=['tweet_id'])
                done_ids.update(df['tweet_id'].unique())
            except Exception as e:
                print(f" Error reading checkpoint file {file}: {e}")

    # Filter tweets that haven't been processed
    pending_df = tweets_df[~tweets_df['id'].isin(done_ids)]

    # Process in batches
    for i in tqdm(range(0, len(pending_df), batch_size), desc="Processing tweet batches"):
        batch = pending_df.iloc[i:i + batch_size]

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(process_single_tweet, row, stock_base_path) for _, row in batch.iterrows()]

            result_dfs = []
            for future in as_completed(futures):
                try:
                    result = future.result()
                    if not result.empty:
                        result_dfs.append(result)
                except Exception as e:
                    print(f" Thread error: {e}")

        if result_dfs:
            batch_result = pd.concat(result_dfs, ignore_index=True)
            out_path = os.path.join(checkpoint_dir, f"checkpoint_{i}_{i + len(batch)}.parquet")
            batch_result.to_parquet(out_path, index=False)
            print(f"Saved checkpoint: {out_path}")
        else:
            print(f"No results for batch {i}–{i + len(batch)}")

In [160]:
import os

checkpoint_dir = "output/checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)

data_path = "/content"
checkpoint_dir = data_path + "/output/checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)

stock_base_path = data_path + "/data/taq/"


In [161]:
from glob import glob
import os
import pandas as pd
from dateutil import parser

# Build (ticker, year) set from parquet directory structure
parquet_paths = glob("siads-699/data/taq/SYM_ROOT=*/year=*")
available_pairs = set()
for path in parquet_paths:
    parts = path.split('/')
    ticker = parts[-2].split('=')[1]
    year = int(parts[-1].split('=')[1])
    available_pairs.add((ticker, year))

# Parse timestamps safely
def safe_parse(ts):
    try:
        return parser.parse(ts)
    except Exception:
        return pd.NaT

tweets_expanded['parsed_timestamp'] = tweets_expanded['timestamp'].apply(safe_parse)
tweets_expanded['year'] = tweets_expanded['parsed_timestamp'].dt.year

# Filter using vectorized merge instead of row-wise apply
available_df = pd.DataFrame(available_pairs, columns=['ticker', 'year'])
filtered = tweets_expanded.merge(available_df, on=['ticker', 'year'], how='inner')

print(f"{len(filtered)} tweet-stock pairs match available data")

6291 tweet-stock pairs match available data


In [162]:
print("Unique handles:", filtered['handle'].unique())
print(filtered['handle'].value_counts())

Unique handles: ['@realDonaldTrump' '@nntaleb' '@paulkrugman' '@elonmusk'
 '@richardbranson' '@JoeBiden' '@SenWarren']
handle
@realDonaldTrump    2761
@elonmusk           2370
@nntaleb             428
@richardbranson      412
@SenWarren           225
@paulkrugman          80
@JoeBiden             15
Name: count, dtype: int64


In [163]:
stock_base_path = "siads-699/data/taq/"
process_tweets_parallel(filtered, stock_base_path, checkpoint_dir)

Processing tweet batches: 100%|██████████| 1/1 [00:04<00:00,  4.85s/it]

No results for batch 0–13


In [164]:
# Debug: Inspect a single tweet-stock pair
row = tweets_expanded.iloc[0]
print(row[['ticker', 'timestamp']])

ticker = row['ticker']
tweet_time = pd.to_datetime(row['timestamp']).tz_localize(None)
year = tweet_time.year

from datetime import timedelta
import glob

files = glob.glob(f"data/taq/SYM_ROOT={ticker}/year={year}/*.parquet")
print("Files found:", files)

if files:
    df = pd.read_parquet(files[0])
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    print("Stock timestamp range:", df['timestamp'].min(), "→", df['timestamp'].max())

    start = tweet_time - timedelta(minutes=10)
    end = tweet_time + timedelta(minutes=10)
    mask = (df['timestamp'] >= start) & (df['timestamp'] <= end)

    print("Tweet ±10 min window:", start, "to", end)
    print("Rows in window:", mask.sum())
else:
    print("No files found for this ticker-year.")

ticker                                   DWAC
timestamp    2021-01-06 23:01:04.935000+00:00
Name: 0, dtype: object
Files found: []
No files found for this ticker-year.


In [ ]:

def combine_checkpoints(checkpoint_dir):
    import glob
    import os
    import pandas as pd

    parquet_files = glob.glob(os.path.join(checkpoint_dir, "*.parquet"))
    if not parquet_files:
        print("No checkpoint files found.")
        return pd.DataFrame()

    dfs = []
    for f in parquet_files:
        try:
            df = pd.read_parquet(f)
            dfs.append(df)
        except Exception as e:
            print(f"Failed to read {f}: {e}")

    combined = pd.concat(dfs, ignore_index=True)
    print(f"Combined shape before deduplication: {combined.shape}")

    combined = combined.drop_duplicates()
    print(f"Shape after deduplication: {combined.shape}")
    return combined


In [165]:
combined = combine_checkpoints(checkpoint_dir)
combined.to_parquet("data/final_combined.parquet", index=False)

Combined shape before deduplication: (815520, 17)
Shape after deduplication: (555780, 17)


In [168]:
combined.to_csv("siads-699/data/final_combined.csv", index=False)